In [1]:
from cvrp_parser import parse_cvrplib_uchoa_2014

In [14]:
# read the file Uchoa_et_al_2014/X-n101-k25.vrp
file_path = 'Uchoa_et_al_2014/X-n101-k25.vrp'
#load the text into a string
with open(file_path, 'r') as file:
    vrp_text = file.read()
# parse the CVRP data
inst  = parse_cvrplib_uchoa_2014(vrp_text)

In [15]:
print(inst.name, inst.n_customers, inst.capacity, inst.edge_weight_type)
print("Depot coords:", inst.coords[0])
print("First customer coords:", inst.coords[1])
print("Distance(0,1):", inst.dist(0,1))

X-n101-k25 100 206 EUC_2D
Depot coords: [365. 689.]
First customer coords: [146. 180.]
Distance(0,1): 554.1137067425782


In [16]:
from constructors import clarke_wright_parallel
from utils import build_distance_matrix, is_feasible
from plot_utils import save_solution, plot_solution_from_file

# Clarke–Wright
cw = clarke_wright_parallel(inst.coords, inst.demand, inst.capacity, inst.edge_weight_type)
print("[CW] routes:", len(cw.routes), "cost:", cw.cost,
      "feasible:", is_feasible(cw.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers))

# 1) Save to .sol
sol_path = save_solution(inst, cw.routes, cw.cost)  # -> solutions/X-n101-k25/X-n101-k25.sol

# 2) Plot from .sol
png_path = plot_solution_from_file(inst, sol_path)    # -> solutions/X-n101-k25/X-n101-k25.png
print("Wrote:", sol_path, "and", png_path)

[CW] routes: 28 cost: 28941.984425178503 feasible: True
Wrote: solutions\X-n101-k25\X-n101-k25.sol and solutions\X-n101-k25\X-n101-k25.png


In [21]:
from local_search import local_search
# Local search
# Use verbose=True for quick logging, or pass your own logger
ls = local_search(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    verbose=True,
    time_limit_sec=10.0,   # stop early, keep best
    improvement_eps=1e-4
)

print(f"BEST cost: {ls.cost:.2f}, routes: {len(ls.routes)}")

15:52:21 [INFO] LS start | routes=28 cost=28941.9844 | time_limit=10.0s | no_gain_passes=2 | impr_eps=0.0001
15:52:21 [INFO]   -> new BEST after 2-opt r=15 i=1 k=2 | best=28891.8841
15:52:21 [INFO] [2-opt] r=15 i=1 k=2 kept
15:52:21 [INFO]   -> new BEST after Or-opt2 r=20 i=0 j=3 | best=28885.9311
15:52:21 [INFO] [Or-opt2] r=20 i=0 j=3 kept
15:52:21 [INFO]   -> new BEST after Swap A=0 ia=0 <-> B=6 ib=0 | best=28885.1959
15:52:21 [INFO] [Swap] A=0 ia=0 <-> B=6 ib=0 kept
15:52:21 [INFO]   -> new BEST after Swap A=5 ia=1 <-> B=14 ib=2 | best=28859.1810
15:52:21 [INFO] [Swap] A=5 ia=1 <-> B=14 ib=2 kept
15:52:21 [INFO]   -> new BEST after 2-opt r=14 i=0 k=1 | best=28843.5708
15:52:21 [INFO] [2-opt] r=14 i=0 k=1 kept
15:52:21 [INFO]   -> new BEST after Swap A=6 ia=2 <-> B=10 ib=1 | best=28826.8898
15:52:21 [INFO] [Swap] A=6 ia=2 <-> B=10 ib=1 kept
15:52:21 [INFO]   -> new BEST after Reloc-inter A=10 i=0 -> B=6 j=3 | best=28825.2188
15:52:21 [INFO] [Reloc-inter] A=10 i=0 -> B=6 j=3 kept
15:5

BEST cost: 28823.47, routes: 28


In [ ]:
# #iterate throughout the folder Uchoa_et_al_2014, and for every .vrp file, do the same as above
# import os
# for filename in os.listdir('Uchoa_et_al_2014'):
#     if filename.endswith('.vrp'):
#         file_path = os.path.join('Uchoa_et_al_2014', filename)
#         with open(file_path, 'r') as file:
#             vrp_text = file.read()
#         inst  = parse_cvrplib_uchoa_2014(vrp_text)

#         print(inst.name, inst.n_customers, inst.capacity, inst.edge_weight_type)

#         # Clarke–Wright
#         cw = clarke_wright_parallel(inst.coords, inst.demand, inst.capacity, inst.edge_weight_type)
#         print("[CW] routes:", len(cw.routes), "cost:", cw.cost,
#               "feasible:", is_feasible(cw.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers))

#         # Local search
#         ls = local_search(
#             coords=inst.coords,
#             demand=inst.demand,
#             Q=inst.capacity,
#             routes=[r[:] for r in cw.routes],
#             improvement_eps=1e-4,
#             edge_weight_type=inst.edge_weight_type,
#             verbose=True,
#             time_limit_sec=10.0,   # stop early, keep best
#         )   
#         print(f"BEST cost: {ls.cost:.2f}, routes: {len(ls.routes)}")
#         print("---------------------------------------------------")

#         #now save the solution and plot it
#         sol_path = save_solution(inst, ls.routes, ls.cost)
#         png_path = plot_solution_from_file(inst, sol_path)
#         print("Wrote:", sol_path, "and", png_path)